REQUIRMENTS

In [ ]:
%pip install scikit-learn
%pip install pandas
%pip install imbalanced-learn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Loading The Dataset 
df = pd.read_csv('./Dataset.tsv', sep='\t', header=0)
df = df.drop(df.columns[[2, 3, 4]], axis=1)

# Divide data between training and test data
X = df['Phrase']
y = df['Cognitive Distortion']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .8)

textclassifier =Pipeline([
   ('vect', CountVectorizer()),
   ('tfidf', TfidfTransformer()),
   ('smote', SMOTE(random_state=0)),
   ('lg', LogisticRegression(multi_class='multinomial', random_state=0))
])

params = {'smote__k_neighbors': [2,3,4,5,6,7,8,9,10, ],
          'lg__penalty': ['l1', 'l2', 'elasticnet', 'none' ]}
 
logistic_regression = GridSearchCV(estimator=textclassifier, param_grid=params, n_jobs=10, cv=10, verbose=5)
logistic_regression.fit(X_train, y_train)

print('Train Accuracy : %.3f'%logistic_regression.best_estimator_.score(X_train, y_train))
print('Test Accuracy : %.3f'%logistic_regression.best_estimator_.score(X_test, y_test))
print('Best Accuracy Through Grid Search : %.3f'%logistic_regression.best_score_)
print('Best Parameters : ',logistic_regression.best_params_)

SAVING MODEL

In [9]:
import pickle

pickle.dump(logistic_regression, open('cognitive_distortion_detector_model.pkl', 'wb'))